In [38]:
import cv2#opencv读取的格式是BGR
import matplotlib.pyplot as plt
import numpy as np 

test

In [39]:
img=cv2.imread('../data/characterTestPattern688.tif')
cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [41]:
img.shape

(688, 688, 3)

In [42]:
def cv_show(name,img):
    cv2.imshow(name,img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

hw1

In [43]:
before=cv2.imread('../data/after.jpg')
after=cv2.imread("../data/before.jpg")

In [44]:
before.shape


(501, 385, 3)

In [45]:
after.shape

(603, 445, 3)

In [24]:
cv_show('11',before)
cv_show('22',after)

In [46]:
# 仿射变换
rows, cols = before.shape[:2]
# 假设我们想要将图像的左上角、右上角和左下角的点映射到新的位置
src_points = np.float32([[0, 0], [cols - 1, 0], [0, rows - 1]])
dst_points = np.float32([[10, 10], [cols - 20, 15], [10, rows - 20]])

# 计算仿射变换矩阵
M = cv2.getAffineTransform(src_points, dst_points)

# 应用仿射变换
affine_transformed_image = cv2.warpAffine(before, M, (cols, rows))

# 显示原图和变换后的图像
cv2.imshow('Original Image', before)
cv2.imshow('Affine Transformed Image', affine_transformed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [47]:
# Convert to grayscale.
img1 = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

In [48]:
# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

In [49]:
# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

In [50]:
# Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

In [51]:
# Match the two sets of descriptors.
matches =list(matcher.match(d1, d2)) 
# Sort matches on the basis of their Hamming distance.
matches.sort(key = lambda x: x.distance)

# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.9)]
no_of_matches = len(matches)

# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

for i in range(len(matches)):
  p1[i, :] = kp1[matches[i].queryIdx].pt
  p2[i, :] = kp2[matches[i].trainIdx].pt

# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpPerspective(before,
                    homography, (width, height))

# Save the output.
cv2.imwrite('output.jpg', transformed_img)

True